In [6]:
#시그모이드 함수 역전파
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1/(1+np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out)*self.out
        return dx

In [8]:
# coding: utf-8
import sys, os
sys.path.append("./code/")
sys.path.append("./code/ch05/")# 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

In [9]:
import inspect

inspect.getsource(TwoLayerNet)

"class TwoLayerNet:\n\n    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):\n        # 가중치 초기화\n        self.params = {}\n        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)\n        self.params['b1'] = np.zeros(hidden_size)\n        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) \n        self.params['b2'] = np.zeros(output_size)\n\n        # 계층 생성\n        self.layers = OrderedDict()\n        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])\n        self.layers['Relu1'] = Relu()\n        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])\n\n        self.lastLayer = SoftmaxWithLoss()\n        \n    def predict(self, x):\n        for layer in self.layers.values():\n            x = layer.forward(x)\n        \n        return x\n        \n    # x : 입력 데이터, t : 정답 레이블\n    def loss(self, x, t):\n        y = self.predict(x)\n        return self.lastLay

In [10]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:4.697005475846991e-10
b1:2.8969722436385237e-09
W2:4.937437229012487e-09
b2:1.4052500382361056e-07


In [11]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train_acc:{}".format(train_acc), "test_acc:{}".format(test_acc))

train_acc:0.12476666666666666 test_acc:0.1218
train_acc:0.9007166666666667 test_acc:0.9055
train_acc:0.9244666666666667 test_acc:0.9274
train_acc:0.9372166666666667 test_acc:0.9376
train_acc:0.9449666666666666 test_acc:0.9429
train_acc:0.9514833333333333 test_acc:0.949
train_acc:0.9568 test_acc:0.9532
train_acc:0.9613333333333334 test_acc:0.9557
train_acc:0.96405 test_acc:0.9594
train_acc:0.9662666666666667 test_acc:0.9614
train_acc:0.9698333333333333 test_acc:0.9651
train_acc:0.9729166666666667 test_acc:0.9676
train_acc:0.9748166666666667 test_acc:0.9668
train_acc:0.97665 test_acc:0.9691
train_acc:0.9773333333333334 test_acc:0.9696
train_acc:0.9777 test_acc:0.9704
train_acc:0.9794 test_acc:0.9705


In [13]:
train_size

60000

In [23]:
x= np.random.uniform(low=-2,high=2, size= 200)

In [22]:
x.ndim

1

In [24]:
print(x)

[-0.12222428 -1.56433733  0.14983593 -0.18989596  1.17605483  0.34582097
 -0.26483655 -1.68776426  1.90375569 -1.26140899 -1.8553708   1.76836906
  1.26198524  1.14223916  0.43681192  1.67190439  1.49249902  1.35817431
 -0.11702724  1.79275483  1.45105439 -0.82838752 -0.66643671  1.94499327
  0.76527822 -0.79557779 -0.24406536  0.92788936  1.2316272   0.86548559
 -1.87614079  0.44906026 -1.62270925 -1.26819498 -0.61193361 -0.7236733
 -1.92462821  0.07751548 -1.93796202  0.53945966 -0.90775971 -0.9823406
  1.0489363  -1.82056404 -0.16854811 -0.45575934 -0.61673115  1.89188943
 -1.69988475  1.4970206  -0.95954801 -0.92459451 -0.55987296 -0.51140427
  1.90537823  1.04600362  0.18039632  1.42964599  0.07809139  1.1890061
 -1.1499849   0.39602523  0.09136903 -0.59298281  1.56089492 -1.94270008
  0.78884571 -0.30270859 -0.55813403 -0.61219163 -0.53890174 -1.97806066
 -0.52988403 -1.04409344  1.89161554  0.94237655 -0.08434745 -0.73508889
 -1.6848092   0.20502615 -1.78628833  0.21058947 -1.64